### Camada Gold - Possíveis Análises

Nessa camada você encontra as análises pedidas no case salvas, mas elas estao sendo feitas em outro notebook separado (case-data-architect/src/analysis)

Neste notebook, fiz uma agregação que pode ser bastante útil em análises de dados - fiz apenas para o yellow como exemplo -, já que ela permite analisar o desempenho dos diferentes vendors ao longo do tempo. Isso é fundamental para monitorar receitas, identificar tendências, sazonalidade e até tomar decisões estratégicas sobre parcerias ou incentivos.


In [0]:
from pyspark.sql.functions import year, month, sum as _sum, col, unix_timestamp, round

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

Out[13]: DataFrame[]

### Lendo os dados da camada silver

In [0]:

df_silver_yellow = spark.read.table("silver.yellow_tripdata")
df_silver_green = spark.read.table("silver.green_tripdata")

### Agregação

Total faturado, percorrido e horas percorridas, por mês, por vendor, dado período.

In [0]:
# Gera coluna de duração da corrida em horas
df_silver_yellow = df_silver_yellow.withColumn(
    "horas_corrida",
    (unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 3600.0
)

# Agregação
df_gold_agg = (
    df_silver_yellow
    .groupBy(
        year("pickup_datetime").alias("year"),
        month("pickup_datetime").alias("month"),
        col("vendor_id").alias("vendor_id")
    )
    .agg(
        round(_sum("total_amount"), 2).alias("total_faturado"),
        round(_sum("trip_distance"), 2).alias("total_percorrido"),
        round(_sum("horas_corrida"), 2).alias("horas_percorridas")
    )
    .orderBy("year", "month", "vendor_id")
)

In [0]:
df_gold_agg.display()

year,month,vendor_id,total_faturado,total_percorrido,horas_percorridas
2023,1,1,1.915540076E7,2393743.8,180691.04
2023,1,2,5.975346966E7,7686131.93,578083.25
2023,2,1,1.843063483E7,2251764.0,176740.97
2023,2,2,5.614787132E7,7025679.19,555909.7
2023,3,1,2.269677576E7,2766875.4,222679.14
2023,3,2,6.731453137E7,8519343.85,679848.6
2023,4,1,2.162960119E7,2661611.3,212876.19
2023,4,2,6.666870254E7,8469269.71,676948.87
2023,5,1,2.324709238E7,2812499.4,237000.86
2023,5,2,7.313308331E7,9223517.52,756635.98


In [0]:
# Caminho para a tabela gold
gold_agg_path = "/FileStore/tables/gold/yellow_agg_vendor_mes"

# Salva como Delta
df_gold_agg.write.format("delta").mode("overwrite").save(gold_agg_path)

# Registra a tabela no schema gold
spark.sql("DROP TABLE IF EXISTS gold.yellow_agg_vendor_mes")
spark.sql(f"CREATE TABLE gold.yellow_agg_vendor_mes USING DELTA LOCATION '{gold_agg_path}'")

Out[11]: DataFrame[]